# Extended Taxi problem 

In this project we will implement the basic Taxi problem  with added gas station and slippery fields.

The taxi moves along the 5x5 grid. 
Its task consists of three phases: 
1. Finding the passenger: going to one of the 4 stations (R, G, Y, B) where the passenger is.
2. Executing the PICKUP action
3. Going towards the destination and executing a DROPOFF action

Additions to the problem:
1. Resource Management (Fuel): Each movement consumes 1 unit of fuel. If the fuel reaches 0, the taxi stops and the episode ends in failure. The agent must learn when to turn to the gas station.
2. Stochasticity (Ice): On certain fields (ice patches), the taxi can slip. This means that the agent gives the command "Go North", but ends up e.g. "East". This teaches the agent to avoid those fields or plan for failure.

# Basic Imports

In [2]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import matplotlib.patches as patches

# Custom Taxi Environment With Fuel

**Our custom environment will inherit from gymnasium.Env that defines the structure all environments must follow**

Fuel: max 16 units (0-15)

Passenger location: 0=R, 1=G, 2=Y, 3=B, 4=in taxi.

Destination: 0=R, 1=G, 2=Y, 3=B

State is defined by **[row(taxi position), column(taxi position), passenger_location, destination, fuel]**

Actions:

    0-> N
    1-> S
    2-> E
    3-> W
    4->PICKUP
    5->DROPOFF
    6->REFUEL

In [ ]:
class TaxiFuelEnv(gym.Env):
    def __init__(self):
        super().__init__()
        self.grid_size = 5
        self.fuel_limit = 15

        # MultiDiscrete - Supports multiple discrete values with multiple axes, used for controller actions
        self.observation_space = spaces.MultiDiscrete([5, 5, 5, 4, 16]) #5 rows and columns, 5 passenger locations, 4 destinations, 16 units of fuel

        self.action_space = spaces.Discrete(7)
        self.locs = [(0,0), (0,4), (4,0), (4,3)] # R, G, Y, B
        self.gas_station = (2,2) 
        self.ice_patches - [(1, 1), (1, 3), (3, 2)] #slippery sections

    
    def reset(self, seed = None):
        super().reset(seed=seed)
        self.taxi_pos = [np.random.randint(0,5), np.random.randint(0,5)] #setting the taxi on a random position in the board
        self.pass_idx = np.random.randint(0,4)  #the passenger is at one of these positions in the beginning: R, Y, B, G
        self.dest_idx = np.random.randint(0,4)  
        self.fuel = self.fuel_limit  #starting with maximum fuel
        return self._get_obs(), {}
    
    def _get_obs(self):
        #convert internal state to observation format
        return (self.taxi_pos[0], self.taxi_pos[1], self.pass_idx, self.dest_idx, self.fuel)
    
    def step(self, action):
        return super().step(action)







# Q Learning Agent

In [ ]:
class QLearningAgent:
    def __init__(self, state_space, action_space):
        ''' 
            state_space - dimensions of the discrete state space
            action_space - possible actions
            alpha - learning rate
            gamma - discount factor
            epsilon - probability of a random action
            Q - Q table
        '''

        self.alpha = 0.1
        self.gamma = 0.99
        # When epsilon = 1, the epsilon-greedy policy chooses actions completely randomly, so it is equivalent to a random policy.
        self.epsilon = 1.0   
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.actions = action_space # [-10, +10]

        # Q table ->  Q[x_i][xdot_i][θ_i][θdot_i][action]
        self.Q = np.zeros(state_space + (len(action_space),))


    def choose_action(self, state):
        '''
            Actions are selected using an epsilon-greedy policy,
            so that exploration and exploitation are balanced.
        '''
        if np.random.rand() < self.epsilon:   
            # Exploration - choose a random action
            return np.random.randint(len(self.actions))
        # Exploitation with probability 1-epsilon, the agent is using his current knowledge to maximize rewards
        return np.argmax(self.Q[state]) 
    
    def update(self, s, a, r, s_next):
        '''Q-values are updated using the Bellman optimality equation
            s - state, tuple
            a - action, 0 or 1
            r - reward
            s_next - next state
        '''
        self.Q[s + (a,)] += self.alpha * (r + self.gamma * np.max(self.Q[s_next]) - self.Q[s + (a,)])